<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará el libro de Arthur C. Doyle de las "Aventuras de Sherlock Holmes" en inglés y "El Arte de la Guerra" de Sun Tzu para generar los embeddings (separados), es decir, que los vectores tendrán la forma en función de como el autor haya utilizado las palabras en su libro.

In [71]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset libros en archivos de texto.

In [72]:
# Descargar la carpeta de dataset
import os
import platform

directory = './dataset/text_corpus'

if os.access(directory, os.F_OK) is False:
    os.makedirs(directory)
if os.access('desafio_2.zip', os.F_OK) is False:
    if platform.system() == 'Windows':
        !curl https://raw.githubusercontent.com/simon-rodriguez/CEIA_procesamiento_lenguaje_natural/main/datasets/text_corpus/desafio_2.zip -O {directory}/desafio_2.zip

    else:
        !wget -O {directory}/desafio_2.zip https://github.com/simon-rodriguez/CEIA_procesamiento_lenguaje_natural/raw/main/datasets/text_corpus/desafio_2.zip

    !unzip -q {directory}/desafio_2.zip -d {directory}

else:
    print("El dataset ya se encuentra descargado")

--2024-09-26 19:43:18--  https://github.com/simon-rodriguez/CEIA_procesamiento_lenguaje_natural/raw/main/datasets/text_corpus/desafio_2.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/simon-rodriguez/CEIA_procesamiento_lenguaje_natural/main/datasets/text_corpus/desafio_2.zip [following]
--2024-09-26 19:43:18--  https://raw.githubusercontent.com/simon-rodriguez/CEIA_procesamiento_lenguaje_natural/main/datasets/text_corpus/desafio_2.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 437883 (428K) [application/zip]
Saving to: ‘./dataset/text_corpus/desafio_2.zip’

./dataset/text_corp 100%[====

In [73]:
# Posibles libros
os.listdir("./dataset/text_corpus")

['el_arte_de_la_guerra.txt',
 'desafio_2.zip',
 '.ipynb_checkpoints',
 'the_adventures_of_sherlock_holmes.txt',
 'the_adventures_of_sherlock_holmes_sentences.txt']

In [74]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
# El libro está preprocesado para que cada línea sea una oración del libro (determinada por el punto al final de la oracion)
df_sherlock = pd.read_csv('./dataset/text_corpus/the_adventures_of_sherlock_holmes_sentences.txt', sep='/n', header=None)
df_sherlock.head()

<ipython-input-74-071a21c1ab2a>:3: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,0
0,I.
1,A SCANDAL IN BOHEMIA I.
2,To Sherlock Holmes she is always the woman.
3,I have seldom heard him mention her under any ...
4,In his eyes she eclipses and predominates the ...


In [75]:
df_arte= pd.read_csv('./dataset/text_corpus/el_arte_de_la_guerra.txt', sep='/n', header=None)
df_arte.head()

<ipython-input-75-b240d15238c1>:1: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,0
0,El Arte de la Guerra
1,Capítulo 1.
2,Sobre la evaluación
3,Sun Tzu dice: la guerra es de vital importanci...
4,No reflexionar seriamente sobre todo lo que le...


In [76]:
print("Cantidad de documentos (Sherlock):", df_sherlock.shape[0])
print("Cantidad de documentos (Arte de la Guerra):", df_arte.shape[0])

Cantidad de documentos (Sherlock): 5877
Cantidad de documentos (Arte de la Guerra): 736


## El Arte de la Guerra

### 1 - Preprocesamiento

In [77]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens_arte = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df_arte[:None].iterrows():
    sentence_tokens_arte.append(text_to_word_sequence(row[0]))

In [78]:
# Demos un vistazo
sentence_tokens_arte[5:7]

[['hay',
  'que',
  'valorarla',
  'en',
  'términos',
  'de',
  'cinco',
  'factores',
  'fundamentales',
  'y',
  'hacer',
  'comparaciones',
  'entre',
  'diversas',
  'condiciones',
  'de',
  'los',
  'bandos',
  'rivales',
  'con',
  'vistas',
  'a',
  'determinar',
  'el',
  'resultado',
  'de',
  'la',
  'guerra'],
 ['el',
  'primero',
  'de',
  'estos',
  'factores',
  'es',
  'la',
  'doctrina',
  'el',
  'segundo',
  'el',
  'tiempo',
  'el',
  'tercero',
  'el',
  'terreno',
  'el',
  'cuarto',
  'el',
  'mando',
  'y',
  'el',
  'quinto',
  'la',
  'disciplina']]

### 2 - Crear los vectores (word2vec)

In [79]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [80]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model_arte = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=400,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [81]:
# Obtener el vocabulario con los tokens
w2v_model_arte.build_vocab(sentence_tokens_arte)

In [82]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model_arte.corpus_count)

Cantidad de docs en el corpus: 736


In [83]:
# Cantidad de palabras/términos encontrados en el corpus
print("Cantidad de términos distintos en el corpus:", len(w2v_model_arte.wv.index_to_key))

Cantidad de términos distintos en el corpus: 374


### 3 - Entrenar embeddings

In [84]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model_arte.train(sentence_tokens_arte,
                 total_examples=w2v_model_arte.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 106914.5859375
Loss after epoch 1: 56698.2265625
Loss after epoch 2: 55810.65625
Loss after epoch 3: 55458.09375
Loss after epoch 4: 55770.1875
Loss after epoch 5: 55848.125
Loss after epoch 6: 55604.8125
Loss after epoch 7: 55374.1875
Loss after epoch 8: 56984.0
Loss after epoch 9: 56945.125
Loss after epoch 10: 56844.1875
Loss after epoch 11: 56088.9375
Loss after epoch 12: 55223.75
Loss after epoch 13: 55625.375
Loss after epoch 14: 54435.5
Loss after epoch 15: 54694.0
Loss after epoch 16: 54979.0
Loss after epoch 17: 54129.875
Loss after epoch 18: 49106.375
Loss after epoch 19: 48513.25
Loss after epoch 20: 48618.625
Loss after epoch 21: 47700.125
Loss after epoch 22: 47603.125
Loss after epoch 23: 46452.875
Loss after epoch 24: 47205.25
Loss after epoch 25: 46108.25
Loss after epoch 26: 46212.375
Loss after epoch 27: 46284.25
Loss after epoch 28: 46565.375
Loss after epoch 29: 45545.375
Loss after epoch 30: 44815.875
Loss after epoch 31: 45306.125
Loss after ep

(278994, 703150)

### 4 - Ensayar

In [85]:
# Palabras que MÁS se relacionan con...:
w2v_model_arte.wv.most_similar(positive=["batalla"], topn=10)

[('vez', 0.7536874413490295),
 ('derrota', 0.7234232425689697),
 ('entrar', 0.7212443351745605),
 ('momento', 0.6954834461212158),
 ('considera', 0.6703100800514221),
 ('después', 0.655949592590332),
 ('kilómetros', 0.6553530097007751),
 ('firmeza', 0.6547974348068237),
 ('combate', 0.6509721875190735),
 ('distancia', 0.6494271159172058)]

In [86]:
# Palabras que MENOS se relacionan con...:
w2v_model_arte.wv.most_similar(negative=["batalla"], topn=10)

[('capítulo', 0.021289438009262085),
 ('prisioneros', -0.13480673730373383),
 ('deben', -0.14050182700157166),
 ('pueden', -0.1446075141429901),
 ('mando', -0.14674396812915802),
 ('entre', -0.15144218504428864),
 ('decir', -0.1705988198518753),
 ('con', -0.19836504757404327),
 ('tener', -0.2127441018819809),
 ('quiere', -0.2177363783121109)]

#### Comentario sobre la palabra "Batalla":

**Similares**

- El momento del día ``[día, noche]``, tienen sentido de estar relacionado con la palabra "Batalla", ya que esta puede ocurrir en cualquier momento.
- El libro habla del "arte de la guerra", por lo que tiene sentido que se relacione la batalla a la palabra "arte".
- ``[Recompensa, Derrota]`` tienen sentido al relacionarse al resultado de una batalla.
- Las demás palabras pueden acompañar fácilmente a "batalla" en diferentes contextos: "las estimaciones realizadas antes de la batalla **indican** victoria", "la victoria sobre los demás obtenida por medio de la batalla no se **considera** una buena victoria", etc.

** Diferentes **
- La palabra "Capítulo" tiene sentido al no relacionarse con ninguna otra palabra más allá del nombre de la sección del libro.
- En el libro la palabra "prisioneros" no se relaciona a las tácticas en las batallas.
- Las demás palabras no son muy específicas y no necesariamente pueden usarse en combinación con "Batalla".

In [87]:
# Palabras que MÁS se relacionan con...:
w2v_model_arte.wv.most_similar(positive=["tropas"], topn=10)

[('movimientos', 0.7488389015197754),
 ('condiciones', 0.7209693193435669),
 ('momento', 0.71792072057724),
 ('sean', 0.717805027961731),
 ('formaciones', 0.705069363117218),
 ('oficiales', 0.6983823180198669),
 ('muchas', 0.679310142993927),
 ('fuerzas', 0.6787497997283936),
 ('puntos', 0.6639254689216614),
 ('éstas', 0.6615324020385742)]

In [88]:
# Palabras que MENOS se relacionan con...:
w2v_model_arte.wv.most_similar(negative=["tropas"], topn=10)

[('capítulo', 0.12329413741827011),
 ('forma', -0.15639089047908783),
 ('lo', -0.17459265887737274),
 ('su', -0.2034849226474762),
 ('tener', -0.20832592248916626),
 ('victoria', -0.20964090526103973),
 ('puede', -0.21548429131507874),
 ('donde', -0.21677356958389282),
 ('les', -0.22275231778621674),
 ('ver', -0.2258344441652298)]

#### Comentario sobre la palabra "Tropas":

**Similares**
- Se puede observar como la mayoría de las palabras ``[movimiento, condiciones, formaciones, oficiales, fuerzas]`` se relacionan fácilmente con la palabra "tropas".
- Las demás palabras ``[fuerzas, puntos, momento]``, también pueden relacionarse con las tropas de un ejército.


**Diferentes**
- Un poco sorprendente es que la "Victoria" no se relacione a las "Tropas"; pero seguramente en el libro se habla más de las tácticas y estrategias, y no de las fuerzas militares.
- Las demás palabras son términos genéricos y conectores que no tienen mucha relación directa.

In [89]:
# Ensayar con una palabra que no está en el vocabulario:
#w2v_model_arte.wv.most_similar(negative=["felicidad"])

Era de esperarse que en un libro sobre estrategia militar, no exista la "Felicidad".

In [90]:
# el método `get_vector` permite obtener los vectores:
vector_general = w2v_model_arte.wv.get_vector("general")
print(vector_general)

[-2.78422982e-01 -8.27897266e-02  2.89034367e-01  2.99643636e-01
  8.92967731e-02 -3.21056068e-01  6.41099811e-02  7.05078021e-02
  6.72486499e-02  4.24132161e-02 -2.11276010e-01  1.02760389e-01
 -2.52803385e-01 -8.10533091e-02  7.26966634e-02 -3.01500767e-01
 -1.48266613e-01  1.07538439e-01  3.14181536e-01 -8.15544054e-02
  1.69714257e-01 -2.33265460e-01  1.70229271e-01 -9.03468728e-02
  1.68228541e-02  2.05941945e-01 -5.82855493e-02  2.12434530e-01
 -1.33478969e-01 -1.88742764e-02  2.68783011e-02 -3.35487157e-01
  1.16752334e-01 -2.11918540e-02  3.34578305e-02 -6.05009124e-02
  7.76425470e-03 -1.66317016e-01 -1.15969986e-01 -1.14088096e-01
 -1.73115581e-01 -1.79555994e-02 -2.87927777e-01 -2.64189877e-02
  2.05164105e-01 -6.94884546e-03 -4.36728671e-02  5.94750196e-02
  2.32194841e-01 -4.85189520e-02  1.90042391e-01 -2.56345719e-01
 -4.42003608e-01 -1.09145753e-01  3.95414867e-02  1.76965564e-01
  7.91345909e-03 -1.72833502e-01 -1.16727732e-01 -5.82075752e-02
  1.08514771e-01 -7.30529

In [91]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model_arte.wv.most_similar(vector_general)

[('general', 1.0),
 ('regla', 0.822921633720398),
 ('esencial', 0.7822731733322144),
 ('armada', 0.7409866452217102),
 ('gobernante', 0.719624936580658),
 ('asunto', 0.7030818462371826),
 ('dirigir', 0.6976878643035889),
 ('causa', 0.6960670351982117),
 ('militares', 0.6925972104072571),
 ('bajo', 0.6774701476097107)]

In [92]:
# Palabras que MÁS se relacionan con...:
w2v_model_arte.wv.most_similar(positive=["general"], topn=10)

[('regla', 0.822921633720398),
 ('esencial', 0.7822731733322144),
 ('armada', 0.7409866452217102),
 ('gobernante', 0.719624936580658),
 ('asunto', 0.7030818462371826),
 ('dirigir', 0.6976878643035889),
 ('causa', 0.6960670351982117),
 ('militares', 0.6925972104072571),
 ('bajo', 0.6774702072143555),
 ('operaciones', 0.6710516810417175)]

Los términos describen fácilmente lo que significa un "General". Además, ambos métodos arrojaron los mismos resultados en términos y los valores de similitud son técnicamente iguales.

### 5 - Visualizar agrupación de vectores

In [93]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [94]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model_arte)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

Se pueden ver grupos de palabras con mucho sentido como:

- ``[generales, expertos, guerreros, ejércitos]``
- ``[armas, militares, operaciones, suministros]``


In [95]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model_arte,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

En el espacio 3D, se pueden observar combinaciones similares al espacio en dos dimensiones anterior. También se pueden ver otras combinaciones como: ``[soldados, suministros]`` o ``[ataque, combate, luchar]``

In [96]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model_arte.wv.vectors)
labels = list(w2v_model_arte.wv.index_to_key)

np.savetxt("vectors_arte.tsv", vectors, delimiter="\t")

with open("labels_arte.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

## Sherlock

Ahora probamos con un libro más largo y con más documentos/lineas y términos.

### 1 - Preprocesamiento

In [97]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens_sherlock = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df_sherlock[:None].iterrows():
    sentence_tokens_sherlock.append(text_to_word_sequence(row[0]))

In [98]:
# Demos un vistazo
sentence_tokens_sherlock[5:7]

[['it',
  'was',
  'not',
  'that',
  'he',
  'felt',
  'any',
  'emotion',
  'akin',
  'to',
  'love',
  'for',
  'irene',
  'adler'],
 ['all',
  'emotions',
  'and',
  'that',
  'one',
  'particularly',
  'were',
  'abhorrent',
  'to',
  'his',
  'cold',
  'precise',
  'but',
  'admirably',
  'balanced',
  'mind']]

### 2 - Crear los vectores (word2vec)

In [99]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [100]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model_sherlock = Word2Vec(min_count=7,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=400,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [101]:
# Obtener el vocabulario con los tokens
w2v_model_sherlock.build_vocab(sentence_tokens_sherlock)

In [102]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model_sherlock.corpus_count)

Cantidad de docs en el corpus: 5877


In [103]:
# Cantidad de palabras/términos encontrados en el corpus
print("Cantidad de términos distintos en el corpus:", len(w2v_model_sherlock.wv.index_to_key))

Cantidad de términos distintos en el corpus: 1597


### 3 - Entrenar embeddings

In [104]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model_sherlock.train(sentence_tokens_sherlock,
                 total_examples=w2v_model_sherlock.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 824053.75
Loss after epoch 1: 609534.0
Loss after epoch 2: 573489.875
Loss after epoch 3: 506207.625
Loss after epoch 4: 493126.5
Loss after epoch 5: 490147.0
Loss after epoch 6: 484933.5
Loss after epoch 7: 466369.75
Loss after epoch 8: 450501.5
Loss after epoch 9: 446772.0
Loss after epoch 10: 444207.0
Loss after epoch 11: 443888.5
Loss after epoch 12: 439212.0
Loss after epoch 13: 438267.0
Loss after epoch 14: 434509.0
Loss after epoch 15: 432022.0
Loss after epoch 16: 428256.0
Loss after epoch 17: 405650.0
Loss after epoch 18: 405321.0
Loss after epoch 19: 400637.0
Loss after epoch 20: 401222.0
Loss after epoch 21: 396630.0
Loss after epoch 22: 394512.0
Loss after epoch 23: 393887.0
Loss after epoch 24: 392186.0
Loss after epoch 25: 389071.0
Loss after epoch 26: 387461.0
Loss after epoch 27: 385399.0
Loss after epoch 28: 386492.0
Loss after epoch 29: 383435.0
Loss after epoch 30: 380743.0
Loss after epoch 31: 379560.0
Loss after epoch 32: 379702.0
Loss after epo

(3104342, 5394700)

### 4 - Ensayar

In [105]:
# Palabras que MÁS se relacionan con...:
w2v_model_sherlock.wv.most_similar(positive=["murder"], topn=10)

[('“where', 0.576553463935852),
 ('“who', 0.538782000541687),
 ('book', 0.49601277709007263),
 ('referred', 0.4879840016365051),
 ('working', 0.4725741147994995),
 ('criminal', 0.47154882550239563),
 ('card', 0.4704204201698303),
 ('banker', 0.4702211320400238),
 ('league', 0.4700000584125519),
 ('irene', 0.46716201305389404)]

In [106]:
# Palabras que MENOS se relacionan con...:
w2v_model_sherlock.wv.most_similar(negative=["murder"], topn=10)

[('sat', 0.009757797233760357),
 ('were', 0.0027708513662219048),
 ('weary', -0.023430386558175087),
 ('sleep', -0.02953529730439186),
 ('waited', -0.02986465021967888),
 ('holmes’', -0.03251326084136963),
 ('morrow', -0.036608558148145676),
 ('seen', -0.04112047702074051),
 ('determined', -0.042432986199855804),
 ('under', -0.044163692742586136)]

#### Comentario sobre la palabra "murder":

**Similares**
- ``[Where, Who, Criminal]`` son de las primeras palabras que se pueden asociar al término "murder" con mucho sentido, en especial en una novela que trata de resolver crímenes.
- Los demás términos no parecen tener mucho que ver con "murder" pero puede que en la historia estén cerca de la palabra.


**Diferentes**
- Claramente ninguna d elas palabras puede asociarse fácilmente con "murder"

In [107]:
# Palabras que MÁS se relacionan con...:
w2v_model_sherlock.wv.most_similar(positive=["london"], topn=10)

[('waterloo', 0.4516870677471161),
 ('east', 0.44881898164749146),
 ('south', 0.44818320870399475),
 ('probable', 0.3789851665496826),
 ('north', 0.3664020299911499),
 ('river', 0.3657698631286621),
 ('main', 0.3653101623058319),
 ('firm', 0.36115124821662903),
 ('baker', 0.36008715629577637),
 ('court', 0.35814809799194336)]

In [108]:
# Palabras que MENOS se relacionan con...:
w2v_model_sherlock.wv.most_similar(negative=["london"], topn=10)

[('remained', 0.048432689160108566),
 ('almost', 0.04181383177638054),
 ('sight', 0.03866637870669365),
 ('after', 0.024102425202727318),
 ('seemed', 0.018902909010648727),
 ('mouth', 0.017376309260725975),
 ('made', 0.01674126461148262),
 ('trouble', 0.0156803447753191),
 ('pay', 0.012983595952391624),
 ('bright', 0.010303530842065811)]

#### Comentario sobre la palabra "Watson":

**Similares**
- Los eventos del libro ocurren en o se refieren constantemente a Londres y lugares de la ciudad, por lo que tiene sentido que las palabras más similares sean de lugares en Londres ``[Waterloo, East, South, North, River, Baker, Court]``.
- Por ejemplo, "Baker Street" es la calle donde vive Sherlock Holmes.


**Diferentes**
- Casi ninguna de los términos tiene sentido en asociación con Londres y el contexto del libro, así que son buenas opciones negativas.

In [109]:
# Ensayar con una palabra que no está en el vocabulario:
# w2v_model.wv.most_similar(negative=["die"])

In [110]:
# el método `get_vector` permite obtener los vectores:
vector_ring = w2v_model_sherlock.wv.get_vector("ring")
print(vector_ring)

[ 2.66139090e-01 -5.55941314e-02  5.30114114e-01 -4.28557694e-01
 -5.64947985e-02  5.98650984e-02  6.60363177e-04 -3.25709358e-02
 -2.51457728e-02  3.32040042e-01  1.27277091e-01 -3.08589220e-01
 -1.18016712e-01  5.41781858e-02 -2.58918315e-01  1.06868729e-01
  5.84232137e-02  6.24724701e-02 -1.00899421e-01 -7.27873147e-02
 -1.92374155e-01  3.19619089e-01  7.79372826e-02  1.34914637e-01
 -1.43033087e-01  4.03938800e-01  9.31097418e-02  1.30083889e-01
 -2.54347712e-01  2.38186747e-01  2.89318204e-01 -1.61282159e-03
  1.22172818e-01 -3.04940104e-01  5.10554900e-03 -4.60175984e-02
  1.76703379e-01  4.16389965e-02  1.22613005e-01 -5.56141920e-02
 -4.02750999e-01  2.72052828e-03 -3.65206867e-01 -2.94951778e-02
  1.30558819e-01  4.34606820e-02  3.37971538e-01  8.52769911e-02
  1.89515904e-01  1.22471750e-01  1.53373405e-01 -7.05350712e-02
 -4.22652692e-01 -7.60423392e-02  1.02386430e-01  2.28531048e-01
  1.23859502e-01 -3.58783066e-01 -3.97322476e-01  1.14974923e-01
 -1.78804100e-02  2.19145

In [111]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model_sherlock.wv.most_similar(vector_ring)

[('ring', 1.0000001192092896),
 ('ryder', 0.5212765336036682),
 ('“holmes', 0.5087913870811462),
 ('envelope', 0.4911746382713318),
 ('raised', 0.47057822346687317),
 ('heavens', 0.4608837962150574),
 ('rope', 0.4524785280227661),
 ('bridge', 0.44794225692749023),
 ('finished', 0.44571903347969055),
 ('suspicion', 0.44034940004348755)]

In [112]:
# Palabras que MÁS se relacionan con...:
w2v_model_sherlock.wv.most_similar(positive=["ring"], topn=10)

[('ryder', 0.5212765336036682),
 ('“holmes', 0.5087913870811462),
 ('envelope', 0.4911746680736542),
 ('raised', 0.47057822346687317),
 ('heavens', 0.46088382601737976),
 ('rope', 0.4524785280227661),
 ('bridge', 0.4479422867298126),
 ('finished', 0.44571903347969055),
 ('suspicion', 0.44034940004348755),
 ('thinking', 0.4375634789466858)]

Se puede ver facilmente que ambos métodos llevan al mismo resultado.

El término "ring" (anillo) puede ser tanto el objeto como "llamar a la puerta/tocar el timbre/llamar por teléfono". Se observa como los términos similares concuerdan con alguno de esos casos.

### 5 - Visualizar agrupación de vectores

In [113]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [114]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model_sherlock)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [115]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model_sherlock,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

**Comentario**:

Es un poco más difícil de encontrar asociaciones en este corpus y en estas dimensiones, sin embargo, se puede encontrar palabras que naturalmente están asociadas entre si: ``[Face, Hands, Head, Eyes]``. Si bien no son términos específicos de la historia, son términos que se asociaron juntos aún cuando el texto no los coloca juntos necesariamente.

In [116]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model_sherlock.wv.vectors)
labels = list(w2v_model_sherlock.wv.index_to_key)

np.savetxt("vectors_sherlock.tsv", vectors, delimiter="\t")

with open("labels_sherlock.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

## Conclusiones Finales

- Una de las ideas era comparar dos libros muy distintos para ver que palabras eran más similares dependiendo del tema; pero no se lograron encontrar palabras que representaran bien este caso (queda como una mejora para comparar con otro libro).
- Las técnicas están interesantes para obtener palabras similares basadas en el contexto en que se utilizan, muchas veces se logran asociaciones incluso cuando no están explícitamente en el texto.
